## **NOTE**:

1. This notebook submission includes only *partial outputs*: fine-tuning, and single dataset dataset evaluation takes signficantly long to complete (approximately 90 minutes for a single fine-tuning run and about 20 minutes per dataset evaluations). To provide structure to this submission notebook - our team compiled our work into a single notebook and intentionally did not run them to perserve computionally resources and time. Where outputs are able to be perserved - we intentionally left to show completion.

2. Models and datasets are intentionally created and collected to perserve history. These should be public and able to be downloaded in if chosen.

## Installation and Imports

In [ ]:
%%capture
!pip install duckduckgo-search langchain-community
!pip install -U ddgs
!pip install datasets==3.6.0
!pip install -U bitsandbytes accelerate
!pip install -U transformers
!pip install trl

In [ ]:
import re
import os
import gc
from enum import Enum
from google.colab import drive, userdata
from google import genai
from google.genai import types
from typing import Union, List, Dict, Any, Optional, Tuple, Union
from functools import partial
import requests
import json
import textwrap
import sys
import pandas as pd
from pydantic import BaseModel, Field
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn.functional import softmax
from scipy.special import softmax
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    TrainingArguments,
    EarlyStoppingCallback,
    PreTrainedModel,
    PreTrainedTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import random
from sklearn.metrics import average_precision_score, confusion_matrix, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from peft import PeftModel, PeftConfig, LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset, Dataset, Features, Value, DatasetDict, concatenate_datasets
from tqdm.auto import tqdm
from trl import SFTTrainer
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.environ['LANGEXTRACT_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GCLOUD_API_KEY'] = userdata.get('GCLOUD_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

gcloud_api_key = os.getenv("GCLOUD_API_KEY")

In [ ]:
drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/experiments/FactGuard/'
os.makedirs(BASE_DIR, exist_ok=True)

Mounted at /content/drive


## Helper Functions

### Project Helpers

In [ ]:
def checkpoint(dataset, path, filename):
  os.makedirs(path, exist_ok=True)
  dataset.to_csv(f"{path}/{filename}")

def load_checkpoint(path, filename):
  return pd.read_csv(f"{path}/{filename}")

def upload_to_huggingface(path, filename, repo_id):
  df = load_checkpoint(path, filename)
  hf_dataset = Dataset.from_pandas(df)
  hf_dataset.push_to_hub(repo_id)

def load_from_huggingface(repo_id, split=None):
  dataset_dict = load_dataset(repo_id)
  if not split:
    return dataset_dict
  dataset = dataset_dict[split]
  return dataset.to_pandas()

In [ ]:
def get_hf_repo_id(username, model_name, hparams):
    short_codes = {'epoch': 'ep', 'batch_size': 'bs', 'learning_rate': 'lr',
                   'lora_rank': 'r', 'lora_alpha': 'a', 'dropout': 'do'}

    param_parts = []
    for k, v in sorted(hparams.items()):
        code = short_codes.get(k, k)
        param_parts.append(f"{code}{v}")

    param_str = "-".join(param_parts)
    clean_model = model_name.split('/')[-1]

    repo_name = f"factguard-{clean_model}-{param_str}"
    return f"{username}/{repo_name}"

### Model Building Helpers

In [ ]:
def get_user_instruction(claim: str,
                         context: Optional[str] = None,
                         enable_model_reasoning: bool = False) -> str:
    VERDICT_RULES = textwrap.dedent("""
    Determine the final verdict:
    * **Yes:** If the claim is fully supported by the Context (if provided) or by external knowledge.
    * **No:** If the claim is false, contradicted, or if there is insufficient evidence to support or deny the claim.
    """).strip()

    if enable_model_reasoning:
        OUTPUT_REQUIREMENT = "Output Requirement: Output the final verdict ('Yes' or 'No') immediately followed by the rationale/evidence."
    else:
        OUTPUT_REQUIREMENT = "Output Requirement: Output the final verdict ('Yes' or 'No') and nothing else."

    instruction_parts = [
        "**Fact-Check and Evidence Verification**" if context else "**Fact-Check**",
        "",
        VERDICT_RULES,
        OUTPUT_REQUIREMENT,
        ""
    ]
    if context:
        instruction_parts.append(f"--- Context ---\n{context}\n")

    instruction_parts.append(f"--- Claim ---\n{claim}")
    instruction_parts.append("--- Verdict and Rationale ---" if enable_model_reasoning else "--- Verdict ---")

    user_instruction = "\n".join(instruction_parts).strip()
    return user_instruction

In [ ]:
def get_model_output(model_output_label: str,
                     rationale: Optional[str] = None,
                     enable_model_reasoning: bool = False) -> str:

    if enable_model_reasoning:
        if rationale is None:
             rationale = "No reasoning provided in the dataset."

        model_output = textwrap.dedent(f"""
        {model_output_label} RATIONALE: {rationale}
        """).strip()

        return model_output
    else:
        model_output = model_output_label.strip()
        return model_output

In [ ]:
def get_probabilities_decoder_only(model: PreTrainedModel, tokenizer: PreTrainedTokenizer, inputs, device):
  with torch.no_grad():
    logits = model(**inputs).logits

  # Extract the logits for the Yes and No tokens
  vocab = tokenizer.get_vocab()
  selected_logits = logits[0, -1, [vocab['True'], vocab['False']]]

  # Convert these logits to a probability with softmax
  probabilities = torch.softmax(selected_logits, dim=0)
  return probabilities


In [ ]:
def get_probabilities_encoder_decoder(
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer, inputs, device):
  decoder_start_token_id = tokenizer.pad_token_id
  if decoder_start_token_id is None:
      raise ValueError(f"Model {model.config.model_type} is Seq2Seq but has no pad_token_id defined for decoder_input_ids.")

  inputs['decoder_input_ids'] = torch.tensor(
      [[decoder_start_token_id]] * inputs['input_ids'].shape[0], # Batch size
      dtype=torch.long,
      device=device
  )
  with torch.no_grad():
      model.eval()
      outputs = model(**inputs)
      logits = outputs.logits

  vocab = tokenizer.get_vocab()
  LOGIT_BIAS_B = 1.45
  yes_token_id = vocab.get('Yes') or vocab.get(' Yes')
  no_token_id = vocab.get('No') or vocab.get(' No')
  raw_yes_logit = logits[0, -1, yes_token_id] # 0 -> -1
  raw_no_logit = logits[0, -1, no_token_id]
  adjusted_yes_logit = raw_yes_logit + LOGIT_BIAS_B
  selected_logits = torch.stack([adjusted_yes_logit, raw_no_logit], dim=0)
  probabilities = torch.softmax(selected_logits, dim=0)
  return probabilities


In [ ]:
def get_probabilities(model: PreTrainedModel, tokenizer: PreTrainedTokenizer, inputs, device):
    model_type = model.config.model_type
    is_seq2seq = model_type in ENCODER_DECODER_MODEL_TYPES
    return (
        get_probabilities_encoder_decoder(model, tokenizer, inputs, device)
        if is_seq2seq
        else get_probabilities_decoder_only(model, tokenizer, inputs, device)
    )

### Evaluation Helper Functions

In [ ]:
def save_experiment_results(df, dataset, model_name, run_type, hparams=None):
    if run_type == 'baseline' or not hparams:
        param_str = "no_params"
    else:
        short_codes = {
            'epoch': 'ep', 'batch_size': 'bs', 'learning_rate': 'lr',
            'lora_rank': 'r', 'lora_alpha': 'a', 'dropout': 'do'
        }

        parts = []
        for k, v in sorted(hparams.items()):
            code = short_codes.get(k, k)
            parts.append(f"{code}{v}")

        param_str = "_".join(parts)

    # Structure: {DATASET}_{MODEL}_{TYPE}_{PARAMS}.parquet
    filename = f"{dataset}_{model_name}_{run_type}_{param_str}.parquet"
    full_path = os.path.join(BASE_DIR, filename)

    # 4. Save
    df.to_parquet(full_path, index=False)
    print(f"Saved: {filename}")

In [ ]:
def load_experiment_result(filepath):
    try:
        df = pd.read_parquet(filepath)
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None

    # Remove path and extension to get: "FEVER_gemma2_finetune_ep3_bs16..."
    filename = os.path.splitext(os.path.basename(filepath))[0]

    # Split the main components (assuming the structure from the previous step)
    # Note: This split logic assumes Model Name does not contain underscores.
    # If your model names have underscores (e.g. 't5_base'), we might need a stricter delimiter.
    parts = filename.split('_')

    dataset = parts[0]
    model_name = parts[1]
    run_type = parts[2]

    # 3. Add basic metadata to DataFrame
    df['dataset'] = dataset
    df['model'] = model_name
    df['run_type'] = run_type

    # The remaining parts of the list are the hyperparameters (e.g., ['ep3', 'bs16', 'lr2e-5'])
    param_parts = parts[3:]

    # Define the mapping from filename short-codes back to full column names
    code_map = {
        'ep': 'epoch',
        'bs': 'batch_size',
        'lr': 'learning_rate',
        'r':  'lora_rank',
        'a':  'lora_alpha',
        'do': 'dropout'
    }

    if 'no_params' in param_parts:
        # Initialize columns with None or NaN for baselines
        for col in code_map.values():
            df[col] = None
    else:
        for item in param_parts:
            # Use Regex to separate the letters (key) from the numbers (value)
            # ^([a-z]+) matches the start letters (e.g., 'lr')
            # (.*)$ matches the rest (e.g., '2e-05' or '0.1')
            match = re.match(r"^([a-z]+)(.*)$", item)

            if match:
                code, value = match.groups()

                col_name = code_map.get(code, code) # fallback to code if not in map

                try:
                    clean_val = float(value)
                    # Convert to int if it's actually an integer (e.g. 3.0 -> 3)
                    if clean_val.is_integer():
                        clean_val = int(clean_val)
                except ValueError:
                    clean_val = value # Keep as string if not a number

                # Assign to the dataframe
                df[col_name] = clean_val

    return df

In [ ]:
def get_probabilities_from_dataset(dataset, pipeline):
  if type(dataset) is not pd.DataFrame:
    dataset = dataset.to_pandas()

  yes_probs = []
  no_probs = []
  for index, row in tqdm(dataset.iterrows(), total=len(dataset)):
    claim = row['claim']
    result = pipeline(claim=claim)

    verdict = result['verdict']
    yes_prob = result['probability_yes']

    yes_probs.append(yes_prob)
    no_probs.append(1 - yes_prob)

  dataset['Yes'] = yes_probs
  dataset['No'] = no_probs
  return dataset

In [ ]:
def display_confusion_matrix(yes_probs, yes_labels, threshold = 0.5):
  predictions = (yes_probs >= threshold).astype(int)
  cm = confusion_matrix(yes_labels, predictions)
  TN, FP, FN, TP = cm.ravel()
  recall_yes = TP / (TP + FN) if (TP + FN) > 0 else 0
  specificity_no = TN / (TN + FP) if (TN + FP) > 0 else 0
  accuracy = (TP + TN) / (TP + FP + FN + TN) if (TP + FP + FN + TN) > 0 else 0
  precision = TP / (TP + FP) if (TP + FP) > 0 else 0
  f1_score = 2 * (precision * recall_yes) / (precision + recall_yes) if (precision + recall_yes) > 0 else 0

  print(f"--- Performance at Threshold: {threshold} ---")
  print(f"Confusion Matrix:\n{cm}")
  print(f"Total Actual Yes's (TP + FN): {TP + FN}")
  print(f"Total Actual No's (TN + FP): {TN + FP}")
  print("---------------------------------------------")
  print(f"Percentage of **Actual Yes's** correct (Recall): {recall_yes:.2%}")
  print(f"Percentage of **Actual No's** correct (Specificity): {specificity_no:.2%}")
  print(f"Overall Accuracy: {accuracy:.2%}")
  print(f"Precision: {precision:.2%}")
  print(f"F1 Score: {f1_score:.2%}")
  print("---------------------------------------------")

## Building LLM

### Model Definitions

In [ ]:
ENCODER_DECODER_MODEL_TYPES = ['t5gemma']
DECODER_ONLY_MODEL_TYPES = ['gemma2']
AVAILABE_MODELS = ['google/flan-t5-large', 'google/t5gemma-2b-2b-ul2-it']
FINETUNED_MODELS = {"rickpereira/FactGuard-Distilled-T5": 'google/t5gemma-2b-2b-ul2-it'}

In [ ]:
class FineTunedModelType(str, Enum):
    T5GEMMA = "t5gemma"
    GEMMA2 = "gemma2"

    def model_repo(self) -> str:
      if self == FineTunedModelType.T5GEMMA:
        return "rickpereira/FactGuard-Distilled-T5"
      elif self == FineTunedModelType.GEMMA2:
        return "rickpereira/FactGuard-Distilled-Decoder"
      else:
        raise ValueError(f"Unknown model type: {self}")

    def base_model(self) -> str:
        if self == FineTunedModelType.T5GEMMA:
            return "google/t5gemma-2b-2b-ul2-it"
        elif self == FineTunedModelType.GEMMA2:
            return "google/gemma-2-2b-it"
        else:
            raise ValueError(f"Unknown model type: {self}")

    def auto_model_class(self) -> str:
        if self == FineTunedModelType.T5GEMMA:
            return AutoModelForSeq2SeqLM
        elif self == FineTunedModelType.GEMMA2:
            return AutoModelForCausalLM

In [ ]:
class EncoderDecoderModelType(str, Enum):
    T5GEMMA = "t5gemma"

    def model_repo(self) -> str:
      return 'google/t5gemma-2b-2b-ul2-it'

    def base_model(self) -> str:
        return "google/t5gemma-2b-2b-ul2-it"

    def auto_model_class(self) -> str:
        return AutoModelForSeq2SeqLM

    def finetuned(self, hparams) -> str:
        return get_hf_repo_id("rickpereira", "t5gemma", hparams)

class DecoderOnlyModelType(str, Enum):
    GEMMA2 = "gemma2"

    def model_repo(self) -> str:
      return 'google/gemma-2-2b-it'

    def base_model(self) -> str:
        return "google/gemma-2-2b-it"

    def auto_model_class(self) -> str:
        return AutoModelForCausalLM

    def finetuned(self, hparams) -> str:
        return get_hf_repo_id("rickpereira", "gemma2", hparams)

In [ ]:
def load_model(model_type: Union[DecoderOnlyModelType, EncoderDecoderModelType, FineTunedModelType],
               use_quantiziation: bool = False,
               generation_config: dict = None) -> Tuple[PreTrainedModel, AutoTokenizer, Dict[str, Any]]:
  model_repo = model_type.model_repo()
  tokenizer_name = model_type.base_model()
  AutoModelClass = model_type.auto_model_class()

  # 4-bit Quantization Config
  nf4_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.float16
  )
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  model = AutoModelClass.from_pretrained(
      model_repo,
      quantization_config=nf4_config if use_quantiziation else None,
      device_map='auto'
  )

  generation_config = {
      "max_new_tokens": 512,
      "temperature": 0.5,
      "do_sample": True,
  }
  if generation_config:
    generation_config.update(generation_config)

  return model, tokenizer, generation_config

## Knowledge Distillation

### Helper Functions

In [ ]:
TARGET_COLUMNS_CAUSAL_LM = ['original_id', 'claim', 'label', 'rationale', 'context']

In [ ]:
def format_text(example: Dict[str, Any]) -> Dict[str, str]:
  claim = example.get('claim', '')
  context = example.get('context', 'No specific context provided.')
  label = example.get('label')
  rationale = example.get('rationale', 'No reasoning provided.')
  if not isinstance(label, bool):
    print(f"Unexpected label type: {type(label)}")
    return

  user_prompt = get_user_instruction(context=context, claim=claim, enable_model_reasoning=False)
  model_output = get_model_output(
      model_output_label="Yes" if label else "No",
      rationale=rationale,
      enable_model_reasoning=False)
  text = (
      f"<start_of_turn>user\n{user_prompt}<end_of_turn>\n"
      f"<start_of_turn>model\n{model_output}<end_of_turn>"
  )

  return {"text": text}

In [ ]:
def prepare_data(dataset_df: pd.DataFrame) -> DatasetDict:
  FINAL_FT_COLUMNS = ['claim', 'context', 'label', 'rationale']
  if dataset_df.empty:
    return DatasetDict({'train': Dataset.from_dict({'text': []})})

  subset = dataset_df[FINAL_FT_COLUMNS].copy()
  formatted_series = subset.apply(format_text, axis=1)
  data_list = [item for item in list(formatted_series) if item is not None]
  dataset_hf = Dataset.from_list(data_list)
  return DatasetDict({'train': dataset_hf})

In [ ]:
def _load_and_align_datasets(fever_repo: str, squad_repo: str) -> DatasetDict:

    fever_distilled_datasets = load_from_huggingface(fever_repo)
    squad_distilled_datasets = load_from_huggingface(squad_repo)

    # FEVER alignment
    fever_aligned = fever_distilled_datasets.remove_columns("label")
    fever_aligned = fever_aligned.rename_column("verdict", "label")
    fever_aligned = fever_aligned.rename_column("original_fever_id", "original_id")
    fever_aligned = fever_aligned.cast_column("original_id", Value("string"))
    fever_aligned = fever_aligned.select_columns(TARGET_COLUMNS_CAUSAL_LM)

    # SQuAD alignment
    squad_aligned = squad_distilled_datasets.rename_column("original_squad_id", "original_id")
    squad_aligned = squad_aligned.select_columns(TARGET_COLUMNS_CAUSAL_LM)

    return DatasetDict(fever=fever_aligned, squad=squad_aligned)

In [ ]:
def _combine_datasets(aligned_datasets: DatasetDict) -> DatasetDict:

    combined_datasets = DatasetDict()
    fever_data = aligned_datasets['fever']
    squad_data = aligned_datasets['squad']

    split_names = set(squad_data.keys()) & set(fever_data.keys())

    for split in split_names:
        combined_datasets[split] = concatenate_datasets([
            squad_data[split],
            fever_data[split]
        ])
    return combined_datasets

In [ ]:
def _prepare_for_seq2seq_tuning(dataset_dict: DatasetDict) -> DatasetDict:

    def seq2seq_mapping_function(example):
        # INPUT_TEXT (Encoder Input): Instruction + Claim + Context
        if not isinstance(example.get('label'), bool):
            print(f"WARNING: Label Value - {example.get('label')} is {type(example.get('label'))}")
            return

        enable_model_reasoning = False
        input_text = get_user_instruction(
            claim=example['claim'],
            context=example['context'],
            enable_model_reasoning=enable_model_reasoning)
        label = "Yes" if example.get('label') else "No"
        target_text = get_model_output(
            model_output_label=label,
            rationale=example['rationale'],
            enable_model_reasoning=enable_model_reasoning)
        return {
            'input_text': input_text,
            'target_text': target_text
        }

    print("Preparing data for Encoder-Decoder (Seq2Seq) fine-tuning...")

    # Apply the mapping function to create the required text columns
    seq2seq_datasets = dataset_dict.map(
        seq2seq_mapping_function,
        remove_columns=TARGET_COLUMNS_CAUSAL_LM,
        batched=False
    )
    return seq2seq_datasets

In [ ]:
def get_distilled_datasets(repo_id: Optional[str] = None, prepare_for_seq2seq: bool = False) -> DatasetDict:
    if repo_id:
        return load_from_huggingface(repo_id)

    aligned_data = _load_and_align_datasets(
        fever_repo="rickpereira/factguard_fever_distilled_datasets",
        squad_repo="rickpereira/factguard_squad_distilled_datasets"
    )
    combined_datasets = _combine_datasets(aligned_data)

    if prepare_for_seq2seq:
        processed_datasets = _prepare_for_seq2seq_tuning(combined_datasets)
        final_dataset = processed_datasets['train']
    else:
        print("Preparing data for Decoder-Only fine-tuning...")
        df = combined_datasets['train'].to_pandas()
        prepared = prepare_data(df)
        final_dataset = prepared['train']

    split_datasets = final_dataset.train_test_split(
        test_size=0.1,
        seed=42
    )
    for split_name in combined_datasets.keys():
        if split_name != 'train' and split_name in split_datasets:
            split_datasets[split_name] = combined_datasets[split_name]

    return split_datasets

## Supervised Fine-Tuning with Distillation Dataset

In [ ]:
def setup_training_arguments(is_seq2seq: bool) -> Any:
    ArgsClass = Seq2SeqTrainingArguments if is_seq2seq else TrainingArguments

    base_args = {
        "num_train_epochs": 1,
        "per_device_train_batch_size": 4,
        "gradient_accumulation_steps": 2,
        "learning_rate": 5e-5,
        "logging_steps": 50,
        "save_strategy": "epoch",
        "eval_strategy": "epoch",
        "load_best_model_at_end": True,
        "metric_for_best_model": "eval_loss",
        "gradient_checkpointing": True,
        "overwrite_output_dir": True,
        "fp16": False,
        "bf16": True if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else False,
        "report_to": "none",
    }

    seq2seq_args = {
        "predict_with_generate": is_seq2seq,
        "dataloader_num_workers": 8,
        "optim": "adamw_bnb_8bit",
    }

    if is_seq2seq:
        all_args = {**base_args, **seq2seq_args}
    else:
        all_args = base_args

    training_args = ArgsClass(**all_args)

    return training_args

### Gemma 2B

In [ ]:
def finetune_decoder_only(train_dataset: Any, eval_dataset: Any, model: PreTrainedModel,
                    tokenizer: PreTrainedTokenizer, use_lora: bool):

    training_args = setup_training_arguments(is_seq2seq=False)

    early_stopping = EarlyStoppingCallback(
        early_stopping_patience=3,
        early_stopping_threshold=0.0,
    )

    peft_config = None
    if use_lora:
        peft_config = LoraConfig(
            r=64,
            lora_alpha=128,
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        )

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
        callbacks=[early_stopping],
    )
    print(f"Starting TRL fine-tuning for Decoder-Only model: {model.config.model_type}")
    trainer.train()
    return trainer

In [ ]:
def finetune(train_dataset: Any, eval_dataset: Any, model: PreTrainedModel,
             tokenizer: PreTrainedTokenizer, hparams: Dict[str, Any], use_lora: bool = False):
    gc.collect()
    torch.cuda.empty_cache()

    model_type = model.config.model_type

    if model_type in ENCODER_DECODER_MODEL_TYPES:
        print(f"Preprocessing data for Encoder-Decoder model: {model_type}")

        eval_dataset = eval_dataset.map(
            lambda x: preprocess_encoder_decoder(x, tokenizer=tokenizer),
            batched=True,
            remove_columns=['input_text', 'target_text'],
            desc="Tokenizing evaluation data"
        )

        train_dataset = train_dataset.map(
            lambda x: preprocess_encoder_decoder(x, tokenizer=tokenizer),
            batched=True,
            remove_columns=['input_text', 'target_text'],
            desc="Tokenizing training data"
        )

        return finetune_encoder_decoder(train_dataset, eval_dataset, model, tokenizer, hparams, use_lora)
    else:
        return finetune_decoder_only(train_dataset, eval_dataset, model, tokenizer, hparams, use_lora)

In [ ]:
model, tokenizer, _ = load_model(DecoderOnlyModelType.GEMMA2, use_quantiziation=True)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
datasets = get_distilled_datasets(prepare_for_seq2seq=False)

README.md:   0%|          | 0.00/462 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.87M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9100 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/430 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/7.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9100 [00:00<?, ? examples/s]

Preparing data for Decoder-Only fine-tuning...


In [ ]:
train_dataset = datasets['train']
eval_dataset = datasets['test']

In [ ]:
trainer = finetune(train_dataset=train_dataset, eval_dataset=eval_dataset, model=model, tokenizer=tokenizer, use_lora=True)

In [ ]:
sft_model = trainer.model
sft_model.push_to_hub("FactGuard-Distilled-Decoder")

#### Run All

In [ ]:
def run_finetuning(all_hparams):
  for hparams in all_hparams:
    trainer = finetune(train_dataset=train_dataset,
                       eval_dataset=eval_dataset,
                       model=model,
                       tokenizer=tokenizer,
                       hparams=hparams,
                       use_lora=True)

    sft_model = trainer.model
    repo_id = get_hf_repo_id("rickpereira", "gemma2", hparams)
    print(f"Pushing model to Hugging Face Hub: {repo_id}")
    sft_model.push_to_hub(repo_id)

In [ ]:
all_hparams = [
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 32,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-5,
        'lora_rank': 8,
        'lora_alpha': 16,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 16,
        'lora_alpha': 16,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 2,
        'batch_size': 4,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.01,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 2,
        'learning_rate': 5e-5,
        'lora_rank': 32,
        'lora_alpha': 64,
        'dropout': 0.01,
        'gradient_checkpointing': True
    },
    {
        'epoch': 3,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 3,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 32,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
]

In [ ]:
run_finetuning(all_hparams)

### T5Gemma

In [ ]:
def finetune_encoder_decoder(train_dataset: Any, eval_dataset: Any, model: PreTrainedModel,
                             tokenizer: PreTrainedTokenizer, use_lora: bool):
    if hasattr(model, "unload"):
        print("Unloading existing PEFT configuration (if present)...")
        model.unload()


    training_args = setup_training_arguments(is_seq2seq=True)

    early_stopping = EarlyStoppingCallback(
        early_stopping_patience=3,
        early_stopping_threshold=0.0,
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    peft_config = None
    if use_lora:
        peft_config = LoraConfig(
            r=64,
            lora_alpha=128,
            lora_dropout=0.05,
            bias="all",
            task_type="SEQ_2_SEQ_LM",
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        )

        model = get_peft_model(model, peft_config)
        if hasattr(model, "enable_input_require_grads"):
            model.enable_input_require_grads()
        elif hasattr(model, "base_model") and hasattr(model.base_model, "enable_input_require_grads"):
             model.base_model.enable_input_require_grads()
        else:
             print("Warning: Could not find enable_input_require_grads method. Proceeding without specific checkpointing fix.")

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[early_stopping],
    )
    print(f"Starting HF fine-tuning for Encoder-Decoder model: {model.config.model_type}")
    trainer.train()
    return trainer

In [ ]:
def preprocess_encoder_decoder(examples: Dict[str, Any], tokenizer: Any, max_input_length: int = 512, max_target_length: int = 128) -> Dict[str, Any]:
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=max_input_length,
        padding="max_length",
        truncation=True
    )

    labels = tokenizer(
        examples["target_text"],
        max_length=max_target_length,
        padding="max_length",
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]

    if tokenizer.pad_token_id is not None:
        model_inputs["labels"] = np.array(model_inputs["labels"])
        model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100
        model_inputs["labels"] = model_inputs["labels"].tolist()

    return model_inputs

In [ ]:
def finetune(train_dataset: Any, eval_dataset: Any, model: PreTrainedModel,
             tokenizer: PreTrainedTokenizer, hparams: Dict[str, Any], use_lora: bool = False):
    gc.collect()
    torch.cuda.empty_cache()

    model_type = model.config.model_type

    if model_type in ENCODER_DECODER_MODEL_TYPES:
        print(f"Preprocessing data for Encoder-Decoder model: {model_type}")

        eval_dataset = eval_dataset.map(
            lambda x: preprocess_encoder_decoder(x, tokenizer=tokenizer),
            batched=True,
            remove_columns=['input_text', 'target_text'],
            desc="Tokenizing evaluation data"
        )

        train_dataset = train_dataset.map(
            lambda x: preprocess_encoder_decoder(x, tokenizer=tokenizer),
            batched=True,
            remove_columns=['input_text', 'target_text'],
            desc="Tokenizing training data"
        )

        return finetune_encoder_decoder(train_dataset, eval_dataset, model, tokenizer, hparams, use_lora)
    else:
        return finetune_decoder_only(train_dataset, eval_dataset, model, tokenizer, hparams, use_lora)

In [ ]:
model, tokenizer, _ = load_model(EncoderDecoderModelType.T5GEMMA, use_quantiziation=True)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/577 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
datasets = get_distilled_datasets(prepare_for_seq2seq=True)

Preparing data for Encoder-Decoder (Seq2Seq) fine-tuning...


Map:   0%|          | 0/30100 [00:00<?, ? examples/s]

In [ ]:
train_dataset = datasets['train']
eval_dataset = datasets['test']

In [ ]:
trainer = finetune(train_dataset=train_dataset, eval_dataset=eval_dataset, model=model, tokenizer=tokenizer, use_lora=True)

In [ ]:
sft_model = trainer.model
sft_model.push_to_hub("FactGuard-Distilled-T5")

#### Run All

In [ ]:
def run_finetuning(all_hparams):
  for hparams in all_hparams:
    trainer = finetune(train_dataset=train_dataset,
                       eval_dataset=eval_dataset,
                       model=model,
                       tokenizer=tokenizer,
                       hparams=hparams,
                       use_lora=True)

    sft_model = trainer.model
    repo_id = get_hf_repo_id("rickpereira", "t5gemma", hparams)
    print(f"Pushing model to Hugging Face Hub: {repo_id}")
    sft_model.push_to_hub(repo_id)

In [ ]:
all_hparams = [
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.05
    },
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-5,
        'lora_rank': 8,
        'lora_alpha': 16,
        'dropout': 0.05
    },
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-5,
        'lora_rank': 16,
        'lora_alpha': 16,
        'dropout': 0.05
    },
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.01
    },
    {
        'epoch': 1,
        'batch_size': 16,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.01
    },
    {
      'epoch': 2,
      'batch_size': 8,
      'learning_rate': 5e-4,
      'lora_rank': 64,
      'lora_alpha': 128,
      'dropout': 0.05
    }
]

In [ ]:
run_finetuning(all_hparams)

## Creating Pipelines

In [ ]:
def non_retrieval_pipeline(claim: str, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, device: str) -> Dict[str, Any]:

    model_type = model.config.model_type
    is_seq2seq = any(t in model_type for t in ENCODER_DECODER_MODEL_TYPES)
    user_instruction = get_user_instruction(claim=claim, context=None)

    if is_seq2seq:
        final_prompt_string = f"{user_instruction}<bos> "
    else:
        final_prompt_string = f"<start_of_turn>user\n{user_instruction}<end_of_turn>\n<start_of_turn>model\n"

    tokenized_inputs = tokenizer(final_prompt_string, return_tensors="pt", truncation=True)

    try:
        if is_seq2seq:
          inputs_dict = {k: v for k, v in tokenized_inputs.items()}
          inputs_dict = {k: v.to(device) for k, v in inputs_dict.items()}
          probabilities_tensor = get_probabilities(model, tokenizer, inputs_dict, device)
        else:
          inputs = tokenized_inputs.to(device)
          probabilities_tensor = get_probabilities(model, tokenizer, inputs, device)

        prob_yes = probabilities_tensor[0].item()
        prob_no = probabilities_tensor[1].item()

        verdict = "Yes" if prob_yes > prob_no else "No"

        return {"verdict": verdict, "probability_yes": prob_yes}

    except Exception as e:
        print(f"Error during logit extraction: {e}")
        return {"verdict": "ERROR", "probability_yes": 0.0}

In [ ]:
def factguard_pipeline(claim: str, search_tool: DuckDuckGoSearchRun, model, tokenizer, device):
    web_context = search_tool.invoke(claim)
    if not web_context:
        web_context = "Search returned no relevant information."

    model_type = model.config.model_type
    is_seq2seq = any(t in model_type for t in ENCODER_DECODER_MODEL_TYPES)
    user_instruction = get_user_instruction(claim=claim, context=web_context)

    if is_seq2seq:
        final_prompt_string = f"{user_instruction}<bos> "
    else:
        final_prompt_string = f"<start_of_turn>user\n{user_instruction}<end_of_turn>\n<start_of_turn>model\n"

    tokenized_inputs = tokenizer(final_prompt_string, return_tensors="pt", truncation=True)
    inputs_dict = {k: v for k, v in tokenized_inputs.items()}
    inputs_dict = {k: v.to(device) for k, v in inputs_dict.items()}

    try:
        probabilities_tensor = get_probabilities(model, tokenizer, inputs_dict, device)
        prob_yes = probabilities_tensor[0].item()
        prob_no = probabilities_tensor[1].item()
        verdict = "Yes" if prob_yes > prob_no else "No"

        return {"verdict": verdict, "probability_yes": prob_yes}

    except Exception as e:
        print(f"Error during logit extraction: {e}")
        return {"verdict": "ERROR", "probability_yes": 0.0}

## Evaluating

In [ ]:
gemma2_hparams = [
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 32,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-5,
        'lora_rank': 8,
        'lora_alpha': 16,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 16,
        'lora_alpha': 16,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 2,
        'batch_size': 4,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.01,
        'gradient_checkpointing': True
    },
    {
        'epoch': 1,
        'batch_size': 2,
        'learning_rate': 5e-5,
        'lora_rank': 32,
        'lora_alpha': 64,
        'dropout': 0.01,
        'gradient_checkpointing': True
    },
    {
        'epoch': 3,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
    {
        'epoch': 3,
        'batch_size': 4,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 32,
        'dropout': 0.05,
        'gradient_checkpointing': True
    },
]

In [ ]:
t5gemma_hparams = [
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-4,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.05
    },
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-5,
        'lora_rank': 8,
        'lora_alpha': 16,
        'dropout': 0.05
    },
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-5,
        'lora_rank': 16,
        'lora_alpha': 16,
        'dropout': 0.05
    },
    {
        'epoch': 1,
        'batch_size': 8,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.01
    },
    {
        'epoch': 1,
        'batch_size': 16,
        'learning_rate': 5e-5,
        'lora_rank': 64,
        'lora_alpha': 128,
        'dropout': 0.01
    },
    {
      'epoch': 2,
      'batch_size': 8,
      'learning_rate': 5e-4,
      'lora_rank': 64,
      'lora_alpha': 128,
      'dropout': 0.05
    }
]

In [ ]:
class EvaluationDatasetType(Enum):
  fever = "fever"
  boolq = "boolq"
  liar = "liar"

  def _load_dataset(self):
    if self == EvaluationDatasetType.fever:
      return load_dataset("rickpereira/FEVER")
    elif self == EvaluationDatasetType.boolq:
      return load_dataset("google/boolq")
    elif self == EvaluationDatasetType.liar:
      return load_dataset("rickpereira/liar")
    else:
      raise ValueError(f"Unknown dataset type: {self}")

  def _prepare_fever(self, dataset):
    TARGET_COLUMNS = ['label', 'claim']
    fever = dataset.select_columns(TARGET_COLUMNS)
    return fever

  def _prepare_boolq(self, dataset):
    TARGET_COLUMNS = ['question', 'answer']
    boolq = dataset.select_columns(TARGET_COLUMNS)
    boolq = boolq.rename_column('question', 'claim')
    boolq= boolq.rename_column('answer', 'label')
    return boolq

  def _prepare_liar(self, dataset):
    TARGET_COLUMNS = ['label', 'claim']
    dataset = dataset.filter(lambda x: x['label'] in ['true', 'false'])
    dataset = dataset.rename_column("statement", "claim")
    liar = dataset.select_columns(TARGET_COLUMNS)
    return liar

  def _prepare(self, dataset):
    if self == EvaluationDatasetType.fever:
      return self._prepare_fever(dataset)
    elif self == EvaluationDatasetType.boolq:
      return self._prepare_boolq(dataset)
    elif self == EvaluationDatasetType.liar:
      return self._prepare_liar(dataset)
    else:
      raise ValueError(f"Unknown dataset type: {self}")

  def load_and_prepare(self):
    dataset = self._load_dataset()
    return self._prepare(dataset)

### FEVER

In [ ]:
fever_datasets = EvaluationDatasetType.fever.load_and_prepare()

README.md:   0%|          | 0.00/657 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211057 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/52765 [00:00<?, ? examples/s]

In [ ]:
test_dataset = fever_datasets['test'].to_pandas()

#### Baseline Results

In [ ]:
baseline = partial(non_retrieval_pipeline,
                   model=model,
                   tokenizer=tokenizer,
                   device="cuda")

In [ ]:
baseline_probs = get_probabilities_from_dataset(test_dataset, baseline)

In [ ]:
yes_labels = baseline_probs.label.apply(lambda x : 1 if x == 'SUPPORTS' else 0)
yes_probs = baseline_probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[Baseline] AU-PRC: {auprc}")

[Baseline] AU-PRC: 0.9475313134649679


In [ ]:
display_confusion_matrix(yes_probs)

--- Performance at Threshold: 0.5 ---
Confusion Matrix:
[[2359  268]
 [2552 4821]]
Total Actual Yes's (TP + FN): 7373
Total Actual No's (TN + FP): 2627
---------------------------------------------
Percentage of **Actual Yes's** correct (Recall): 65.39%
Percentage of **Actual No's** correct (Specificity): 89.80%
Overall Accuracy: 71.80%
Precision: 94.73%
F1 Score: 77.37%
---------------------------------------------


In [ ]:
save_experiment_results(
    baseline_probs,
    dataset='FEVER',
    model_name='t5gemma',
    run_type='baseline'
)

#### FactGuard - LLM Eval

In [ ]:
sft_model, tokenizer, _ = load_model(FineTunedModelType.T5GEMMA, use_quantiziation=True)
sft_model = sft_model.to("cuda")

config.json: 0.00B [00:00, ?B/s]

adapter_config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/767M [00:00<?, ?B/s]

In [ ]:
factguard_llm = partial(non_retrieval_pipeline,
                        model=sft_model,
                        tokenizer=tokenizer, device="cuda")

In [ ]:
probs = get_probabilities_from_dataset(test_dataset, factguard_llm)

In [ ]:
yes_labels = probs.label.apply(lambda x : 1 if x == 'SUPPORTS' else 0)
yes_probs = probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[SFT - LLM ONLY] AU-PRC: {auprc}")

[SFT - LLM ONLY] AU-PRC: 0.9739125974104663


In [ ]:
display_confusion_matrix(yes_probs)

--- Performance at Threshold: 0.5 ---
Confusion Matrix:
[[2540   87]
 [2621 4752]]
Total Actual Yes's (TP + FN): 7373
Total Actual No's (TN + FP): 2627
---------------------------------------------
Percentage of **Actual Yes's** correct (Recall): 64.45%
Percentage of **Actual No's** correct (Specificity): 96.69%
Overall Accuracy: 72.92%
Precision: 98.20%
F1 Score: 77.83%
---------------------------------------------


In [ ]:
save_experiment_results(
    probs,
    dataset='FEVER',
    model_name='t5gemma',
    run_type='finetune',
    hparams=t5gemma_hparams[0]
)

#### FactGuard - RAG Eval

In [ ]:
factguard = partial(factguard_pipeline,
                    search_tool=DuckDuckGoSearchRun(),
                    model=sft_model,
                    tokenizer=tokenizer, device="cuda")

In [ ]:
probs = get_probabilities_from_dataset(test_dataset, factguard)

In [ ]:
yes_labels = probs.label.apply(lambda x : 1 if x == 'SUPPORTS' else 0)
yes_probs = probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[SFT - RAG] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs)

In [ ]:
save_experiment_results(
    probs,
    dataset='FEVER',
    model_name='t5gemma',
    run_type='rag',
    hparams=t5gemma_hparams[0]
)

#### Run All

In [ ]:
def evaluate_fever(hparams, sft_model, tokenizer, model_name):
  print(f"Evals: {hparams}")
  fever_datasets = EvaluationDatasetType.fever.load_and_prepare()
  test_dataset = fever_datasets['test'].to_pandas().sample(n=1000, random_state=42)

  factguard_llm = partial(non_retrieval_pipeline,
                        model=sft_model,
                        tokenizer=tokenizer, device="cuda")
  probs = get_probabilities_from_dataset(test_dataset, factguard_llm)
  yes_labels = probs.label.apply(lambda x : 1 if x == 'SUPPORTS' else 0)
  yes_probs = probs['Yes']
  auprc = average_precision_score(yes_labels, yes_probs)
  print(f"[SFT - LLM ONLY] AU-PRC: {auprc}")
  display_confusion_matrix(yes_probs, yes_labels)
  save_experiment_results(
      probs,
      dataset='FEVER',
      model_name=model_name,
      run_type='finetune',
      hparams=hparams
  )

  factguard = partial(factguard_pipeline,
                    search_tool=DuckDuckGoSearchRun(),
                    model=sft_model,
                    tokenizer=tokenizer, device="cuda")
  probs = get_probabilities_from_dataset(test_dataset, factguard)
  yes_labels = probs.label.apply(lambda x : 1 if x == 'SUPPORTS' else 0)
  yes_probs = probs['Yes']
  auprc = average_precision_score(yes_labels, yes_probs)
  print(f"[SFT RAG] AU-PRC: {auprc}")
  display_confusion_matrix(yes_probs, yes_labels)
  save_experiment_results(
      probs,
      dataset='FEVER',
      model_name=model_name,
      run_type='rag',
      hparams=hparams
  )

In [ ]:
def evaluate_fever_all(all_hparams, model_type):
  if model_type == EncoderDecoderModelType.T5GEMMA:
    model_name = "t5gemma"
  elif model_type == DecoderOnlyModelType.GEMMA2:
    model_name = "gemma2b"
  else:
    return ValueError("Unknonw Model Type")

  for hparams in all_hparams:
    sft_model, tokenizer = load_model(
        model_type=model_type,
        use_quantiziation=True, finetuned=True, hparams=hparams
    )
    sft_model = sft_model.to("cuda")
    evaluate_fever(hparams, sft_model, tokenizer, model_name)

#### Evaluating Both Models

In [ ]:
evaluate_fever_all(t5gemma_hparams, EncoderDecoderModelType.T5GEMMA)
evaluate_fever_all(gemma2_hparams, DecoderOnlyModelType.GEMMA2)

### BOOLQ

In [ ]:
boolq_datasets = EvaluationDatasetType.boolq.load_and_prepare()

In [ ]:
boolq_test_dataset = boolq_datasets['validation'].to_pandas()

#### Baseline Results

In [ ]:
model, tokenizer, generation_config = load_model(EncoderDecoderModelType.T5GEMMA)

In [ ]:
baseline = partial(non_retrieval_pipeline,
                   model=model,
                   tokenizer=tokenizer,
                   device="cuda")

In [ ]:
baseline_probs = get_probabilities_from_dataset(boolq_test_dataset, baseline)

In [ ]:
yes_labels = baseline_probs.label.apply(lambda x : 1 if x else 0)
yes_probs = baseline_probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[Baseline] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs)

In [ ]:
save_experiment_results(
    baseline_probs,
    dataset='boolq',
    model_name='t5gemma',
    run_type='baseline'
)

#### FactGuard - LLM Eval

In [ ]:
sft_model, tokenizer, _ = load_model(FineTunedModelType.T5GEMMA, use_quantiziation=True)
sft_model = sft_model.to("cuda")

In [ ]:
factguard_llm = partial(non_retrieval_pipeline,
                        model=sft_model,
                        tokenizer=tokenizer, device="cuda")

In [ ]:
probs = get_probabilities_from_dataset(boolq_test_dataset, factguard_llm)

In [ ]:
yes_labels = probs.label.apply(lambda x : 1 if x else 0)
yes_probs = probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[SFT - LLM ONLY] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs, yes_labels)

In [ ]:
save_experiment_results(
    probs,
    dataset='boolq',
    model_name='t5gemma',
    run_type='finetune',
    hparams=t5gemma_hparams[0]
)

#### FactGuard - RAG Eval

In [ ]:
factguard = partial(factguard_pipeline,
                    search_tool=DuckDuckGoSearchRun(),
                    model=sft_model,
                    tokenizer=tokenizer, device="cuda")

In [ ]:
probs = get_probabilities_from_dataset(boolq_test_dataset, factguard)

In [ ]:
yes_labels = probs.label.apply(lambda x : 1 if x else 0)
yes_probs = probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[SFT - RAG] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs, yes_labels)

In [ ]:
save_experiment_results(
    probs,
    dataset='boolq',
    model_name='t5gemma',
    run_type='rag',
    hparams=t5gemma_hparams[0]
)

#### Run All

In [ ]:
def evaluate_boolq(hparams, sft_model, tokenizer, model_name):
  print(f"Evals: {hparams}")
  factguard_llm = partial(non_retrieval_pipeline,
                        model=sft_model,
                        tokenizer=tokenizer, device="cuda")
  probs = get_probabilities_from_dataset(boolq_test_dataset, factguard_llm)
  yes_labels = probs.label.apply(lambda x : 1 if x else 0)
  yes_probs = probs['Yes']
  auprc = average_precision_score(yes_labels, yes_probs)
  print(f"[SFT - LLM ONLY] AU-PRC: {auprc}")
  display_confusion_matrix(yes_probs, yes_labels)
  save_experiment_results(
      probs,
      dataset='boolq',
      model_name=model_name,
      run_type='finetune',
      hparams=hparams
  )
  # RAG
  factguard = partial(factguard_pipeline,
                      search_tool=DuckDuckGoSearchRun(),
                      model=sft_model,
                      tokenizer=tokenizer, device="cuda")
  probs = get_probabilities_from_dataset(boolq_test_dataset, factguard)
  yes_labels = probs.label.apply(lambda x : 1 if x else 0)
  yes_probs = probs['Yes']
  auprc = average_precision_score(yes_labels, yes_probs)
  print(f"[SFT - RAG] AU-PRC: {auprc}")
  display_confusion_matrix(yes_probs, yes_labels)
  save_experiment_results(
      probs,
      dataset='boolq',
      model_name=model_name,
      run_type='rag',
      hparams=hparams
  )

In [ ]:
def evaluate_boolq_all(all_hparams, model_type):
  if model_type == EncoderDecoderModelType.T5GEMMA:
    model_name = "t5gemma"
  elif model_type == DecoderOnlyModelType.GEMMA2:
    model_name = "gemma2b"
  else:
    return ValueError("Unknonw Model Type")

  for hparams in all_hparams:
    sft_model, tokenizer = load_model(
        model_type=model_type,
        use_quantiziation=True, finetuned=True, hparams=hparams
    )
    sft_model = sft_model.to("cuda")
    evaluate_boolq(hparams, sft_model, tokenizer)

#### Evaluating Both Models

In [ ]:
evaluate_boolq_all(t5gemma_hparams, EncoderDecoderModelType.T5GEMMA)
evaluate_boolq_all(gemma2_hparams, DecoderOnlyModelType.GEMMA2)

### LIAR

In [ ]:
liar_datasets = EvaluationDatasetType.liar.load_and_prepare()

In [ ]:
liar_dataset = liar_datasets['test'].to_pandas()

#### Baseline Results

In [ ]:
model, tokenizer, generation_config = load_model(EncoderDecoderModelType.T5GEMMA)

In [ ]:
baseline = partial(non_retrieval_pipeline,
                   model=model,
                   tokenizer=tokenizer,
                   device="cuda")

In [ ]:
baseline_probs = get_probabilities_from_dataset(liar_dataset, baseline)

In [ ]:
yes_labels = baseline_probs.label.apply(lambda x : 1 if x == 'true' else 0)
yes_probs = baseline_probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[Baseline] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs)

In [ ]:
save_experiment_results(
    baseline_probs,
    dataset='liar',
    model_name='t5gemma',
    run_type='baseline'
)

#### FactGuard - LLM Results

In [ ]:
sft_model, tokenizer, _ = load_model(FineTunedModelType.T5GEMMA, use_quantiziation=True)
sft_model = sft_model.to("cuda")

In [ ]:
factguard_llm = partial(non_retrieval_pipeline,
                        model=sft_model,
                        tokenizer=tokenizer, device="cuda")

In [ ]:
probs = get_probabilities_from_dataset(liar_dataset, factguard_llm)

In [ ]:
yes_labels = probs.label.apply(lambda x : 1 if x == 'true' else 0)
yes_probs = probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[SFT - LLM Only] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs, yes_labels)

In [ ]:
save_experiment_results(
    probs,
    dataset='liar',
    model_name='t5gemma',
    run_type='finetune',
    hparams=t5gemma_hparams[0]
)

#### FactGuard - RAG Eval

In [ ]:
factguard = partial(factguard_pipeline,
                    search_tool=DuckDuckGoSearchRun(),
                    model=sft_model,
                    tokenizer=tokenizer, device="cuda")

In [ ]:
probs = get_probabilities_from_dataset(liar_dataset, factguard)

In [ ]:
yes_labels = probs.label.apply(lambda x : 1 if x == 'true' else 0)
yes_probs = probs['Yes']
auprc = average_precision_score(yes_labels, yes_probs)
print(f"[SFT - RAG] AU-PRC: {auprc}")

In [ ]:
display_confusion_matrix(yes_probs, yes_labels)

In [ ]:
save_experiment_results(
    probs,
    dataset='liar',
    model_name='t5gemma',
    run_type='rag',
    hparams=t5gemma_hparams[0]
)

#### Run All

In [ ]:
def evaluate_liar(hparams, sft_model, tokenizer, model_name):
  factguard_llm = partial(non_retrieval_pipeline,
                        model=sft_model,
                        tokenizer=tokenizer, device="cuda")
  probs = get_probabilities_from_dataset(liar_dataset, factguard_llm)
  yes_labels = probs.label.apply(lambda x : 1 if x == 'true' else 0)
  yes_probs = probs['Yes']
  auprc = average_precision_score(yes_labels, yes_probs)
  print(f"[SFT LLM Only] AU-PRC: {auprc}")
  display_confusion_matrix(yes_probs, yes_labels)
  save_experiment_results(
      probs,
      dataset='liar',
      model_name=model_name,
      run_type='finetune',
      hparams=hparams
  )

  # RAG
  factguard = partial(factguard_pipeline,
                      search_tool=DuckDuckGoSearchRun(),
                      model=sft_model,
                      tokenizer=tokenizer, device="cuda")
  probs = get_probabilities_from_dataset(liar_dataset, factguard)
  yes_labels = probs.label.apply(lambda x : 1 if x == 'true' else 0)
  yes_probs = probs['Yes']
  auprc = average_precision_score(yes_labels, yes_probs)
  print(f"[SFT RAG] AU-PRC: {auprc}")
  display_confusion_matrix(yes_probs, yes_labels)
  save_experiment_results(
      probs,
      dataset='liar',
      model_name=model_name,
      run_type='rag',
      hparams=hparams
  )

In [ ]:
def evaluate_liar_all(all_hparams, model_type):
  if model_type == EncoderDecoderModelType.T5GEMMA:
    model_name = "t5gemma"
  elif model_type == DecoderOnlyModelType.GEMMA2:
    model_name = "gemma2b"
  else:
    return ValueError("Unknonw Model Type")

  for hparams in all_hparams:
    gc.collect()
    torch.cuda.empty_cache()
    sft_model, tokenizer = load_model(
        model_type=model_type,
        use_quantiziation=True, finetuned=True, hparams=hparams
    )
    sft_model = sft_model.to("cuda")
    evaluate_liar(hparams, sft_model, tokenizer, model_name)
    # Clear
    del sft_model
    del tokenizer

#### Evaluating Both Models

In [ ]:
evaluate_liar_all(t5gemma_hparams, EncoderDecoderModelType.T5GEMMA)
evaluate_liar_all(gemma2_hparams, DecoderOnlyModelType.GEMMA2)

## AU-PRC

In [ ]:
def convert_labels_to_int(series):
    s = series.copy()
    if s.dtype == object:
        s = s.str.upper().replace({'SUPPORTS': 1, 'REFUTES': 0, 'TRUE': 1, 'FALSE': 0})
        s = pd.to_numeric(s, errors='coerce')
    elif s.dtype == bool:
        s = s.astype(int)

    s = s.dropna()
    return s.astype(int)

In [ ]:
baseline_fever_name = "FEVER_t5gemma_baseline_no_params.parquet"
baseline_boolq_name = "boolq_t5gemma_baseline_no_params.parquet"
baseline_liar_name = "liar_t5gemma_baseline_no_params.parquet"

finetune_fever_name = "FEVER_t5gemma_finetune_bs8_do0.05_ep1_lr5e-05_a16_r16.parquet"
finetune_boolq_name = "boolq_t5gemma_finetune_bs8_do0.05_ep1_lr5e-05_a16_r16.parquet"
finetune_liar_name = "liar_t5gemma_finetune_bs8_do0.05_ep1_lr5e-05_a16_r16.parquet"

rag_fever_name = "FEVER_t5gemma_rag_bs8_do0.05_ep1_lr5e-05_a16_r16.parquet"
rag_boolq_name = "boolq_t5gemma_rag_bs8_do0.05_ep1_lr5e-05_a16_r16.parquet"
rag_liar_name = "liar_t5gemma_rag_bs8_do0.05_ep1_lr5e-05_a16_r16.parquet"

baseline_fever = load_experiment_result(os.path.join(BASE_DIR, baseline_fever_name))
baseline_boolq = load_experiment_result(os.path.join(BASE_DIR, baseline_boolq_name))
baseline_liar = load_experiment_result(os.path.join(BASE_DIR, baseline_liar_name))

finetune_fever = load_experiment_result(os.path.join(BASE_DIR, finetune_fever_name))
finetune_boolq = load_experiment_result(os.path.join(BASE_DIR, finetune_boolq_name))
finetune_liar = load_experiment_result(os.path.join(BASE_DIR, finetune_liar_name))

rag_fever = load_experiment_result(os.path.join(BASE_DIR, rag_fever_name))
rag_boolq = load_experiment_result(os.path.join(BASE_DIR, rag_boolq_name))
rag_liar = load_experiment_result(os.path.join(BASE_DIR, rag_liar_name))

In [ ]:
dataframes = {
    'FEVER': {'Baseline': baseline_fever, 'Finetune': finetune_fever, 'RAG': rag_fever},
    'BoolQ': {'Baseline': baseline_boolq, 'Finetune': finetune_boolq, 'RAG': rag_boolq},
    'LIAR': {'Baseline': baseline_liar, 'Finetune': finetune_liar, 'RAG': rag_liar}
}

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
ax.plot([0, 1], [0, 1],
        linestyle='--', color='gray', label='Random Guess Baseline (AUC = 0.500)', linewidth=1.5, zorder=1)


colors = {'Baseline': 'blue', 'Finetune': 'green', 'RAG': 'red'}
linestyles = {'FEVER': '-', 'BoolQ': '--', 'LIAR': ':'}

for dataset in ['FEVER', 'BoolQ', 'LIAR']:
    for model, df in dataframes[dataset].items():
        y_true = convert_labels_to_int(df['label'])
        aligned_data = pd.DataFrame({'y_true': y_true, 'y_scores': df.loc[y_true.index, 'Yes']}).dropna()
        y_true_aligned = aligned_data['y_true']
        y_scores_aligned = aligned_data['y_scores']
        fpr, tpr, _ = roc_curve(y_true_aligned, y_scores_aligned)
        roc_auc = auc(fpr, tpr)
        label = f'{dataset} - {model} (AUC = {roc_auc:.3f})'
        ax.plot(fpr, tpr,
                color=colors[model],
                linestyle=linestyles[dataset],
                label=label,
                linewidth=2,
                zorder=2)

ax.set_title('AU-ROC: T5 Gemma Models Across Datasets', fontsize=18)
ax.set_xlabel('False Positive Rate', fontsize=14)
ax.set_ylabel('True Positive Rate', fontsize=14)
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, linestyle='dotted', alpha=0.6)
plt.tight_layout()

In [ ]:
baseline_fever_name = "FEVER_gemma2b_baseline_no_params.parquet"
baseline_boolq_name = "boolq_gemma2b_baseline_no_params.parquet"
baseline_liar_name = "liar_gemma2b_baseline_no_params.parquet"

finetune_fever_name = "FEVER_gemma2b_finetune_bs4_do0.05_ep3_gradient_checkpointingTrue_lr0.0005_a32_r64.parquet"
finetune_boolq_name = "boolq_gemma2b_finetune_bs4_do0.05_ep3_gradient_checkpointingTrue_lr0.0005_a32_r64.parquet"
finetune_liar_name = "liar_gemma2b_finetune_bs4_do0.05_ep3_gradient_checkpointingTrue_lr0.0005_a32_r64.parquet"

rag_fever_name = "FEVER_gemma2b_rag_bs4_do0.05_ep3_gradient_checkpointingTrue_lr0.0005_a32_r64.parquet"
rag_boolq_name = "boolq_gemma2b_rag_bs4_do0.05_ep3_gradient_checkpointingTrue_lr0.0005_a32_r64.parquet"
rag_liar_name = "liar_gemma2b_rag_bs4_do0.05_ep3_gradient_checkpointingTrue_lr0.0005_a32_r64.parquet"

baseline_fever = load_experiment_result(os.path.join(BASE_DIR, baseline_fever_name))
baseline_boolq = load_experiment_result(os.path.join(BASE_DIR, baseline_boolq_name))
baseline_liar = load_experiment_result(os.path.join(BASE_DIR, baseline_liar_name))

finetune_fever = load_experiment_result(os.path.join(BASE_DIR, finetune_fever_name))
finetune_boolq = load_experiment_result(os.path.join(BASE_DIR, finetune_boolq_name))
finetune_liar = load_experiment_result(os.path.join(BASE_DIR, finetune_liar_name))

rag_fever = load_experiment_result(os.path.join(BASE_DIR, rag_fever_name))
rag_boolq = load_experiment_result(os.path.join(BASE_DIR, rag_boolq_name))
rag_liar = load_experiment_result(os.path.join(BASE_DIR, rag_liar_name))

In [ ]:
dataframes = {
    'FEVER': {'Baseline': baseline_fever, 'Finetune': finetune_fever, 'RAG': rag_fever},
    'BoolQ': {'Baseline': baseline_boolq, 'Finetune': finetune_boolq, 'RAG': rag_boolq},
    'LIAR': {'Baseline': baseline_liar, 'Finetune': finetune_liar, 'RAG': rag_liar}
}

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
ax.plot([0, 1], [0, 1],
        linestyle='--', color='gray', label='Random Guess Baseline (AUC = 0.500)', linewidth=1.5, zorder=1)


colors = {'Baseline': 'blue', 'Finetune': 'green', 'RAG': 'red'}
linestyles = {'FEVER': '-', 'BoolQ': '--', 'LIAR': ':'}

for dataset in ['FEVER', 'BoolQ', 'LIAR']:
    for model, df in dataframes[dataset].items():
        y_true = convert_labels_to_int(df['label'])
        aligned_data = pd.DataFrame({'y_true': y_true, 'y_scores': df.loc[y_true.index, 'Yes']}).dropna()
        y_true_aligned = aligned_data['y_true']
        y_scores_aligned = aligned_data['y_scores']
        fpr, tpr, _ = roc_curve(y_true_aligned, y_scores_aligned)
        roc_auc = auc(fpr, tpr)
        label = f'{dataset} - {model} (AUC = {roc_auc:.3f})'
        ax.plot(fpr, tpr,
                color=colors[model],
                linestyle=linestyles[dataset],
                label=label,
                linewidth=2,
                zorder=2)

ax.set_title('AU-ROC: Gemma 2 Models Across Datasets', fontsize=18)
ax.set_xlabel('False Positive Rate', fontsize=14)
ax.set_ylabel('True Positive Rate', fontsize=14)
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, linestyle='dotted', alpha=0.6)
plt.tight_layout()